# <center> **Proyecto Final IA**
**INTEGRANTES:**
>*    Pedro López V.
>*    Ignacio Reyes V.
    
    
**Se procedera a desarrollar una aplicacion donde se aplicara una serie de algoritmos de Machine Learning sobre un dataset que permite predecir el valor de Bitcoin a lo largo de tiempo, con el fin de comparar los distintos metodos en terminos de desempeño.**

## <center> Cargamos librerias a utilizar

In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
import warnings
from pandas import DataFrame
from pandas import concat

warnings.filterwarnings("ignore")
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn import neighbors
from numpy import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestRegressor

## <center> Cargamos dataset seleccionado

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df = pd.read_csv('/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')
print('Esta base de datos consta de (filas,columnas): ', df.shape,'\n')

In [ ]:
df.info()

## <center> Caracteristicas de cada columna
* **Timestamp :** Hora de inicio de la ventana de tiempo (ventana de 60 segundos)
* **Open:** Precio de apertura en la ventana de hora de inicio 
* **High :** Precio alto dentro de la ventana de tiempo
* **Low :** Precio bajo dentro de la ventana de tiempo
* **Close :** Precio de cierre al final de la ventana de tiempo
* **Volume_(BTC) :** Volumen de BTC negociado en esta ventana
* **Volume_(Currency) :** Volumen de la moneda correspondiente negociada en esta ventana
* **Weighted_Price :** VWAP- Precio promedio ponderado por volumen

 **Se puede observar que son casi 5 Millones de datos , esto debido a que la muestra esta tomada cada muy poco tiempo, por lo que acontinuacion se procedera a 
resamplear los datos, haciendo un promedio de los valores dentro de un día. Ademas se procede a eliminar las columnas de Open, High, Low y Close debido a que 
si se saca un promedio total por día, estos 4 valores seran los mismos, tambien se procede a limpiar la data de los valores nulos.**

In [ ]:
df.index = pd.to_datetime(df['Timestamp'],unit='s')
df=df.drop(['Timestamp'],axis=1)
df=df.resample('D').mean()
df=df.dropna()
df=df.drop(['Open','High','Low','Close'],axis=1)

## <center> Correlacion y Boxplot
**A continuacion se crearan funciones para realizar la correlacion de las variables y que se grafiquen los boxplots.**

In [ ]:
def grafico_bloxplot(data):
    plt.rcParams['figure.figsize']=10,5
    vs3=sns.boxplot(data=data,orient='v',color="Green",palette="Set2")
    plt.title("'Identificación de outliers en la base de datos'")
    plt.show()
def correlacion1(data,metodo):
    plt.rcParams['figure.figsize']=10,5
    correlacion=data.corr(method=metodo,min_periods=10)
    sns.heatmap(correlacion, cmap='YlGnBu', annot=True, fmt='.2f', vmin=0)
    plt.show() 

In [ ]:
correlacion1(df,'pearson')

In [ ]:
grafico_bloxplot(df['Volume_(BTC)'])
grafico_bloxplot(df['Volume_(Currency)'])
grafico_bloxplot(df['Weighted_Price'])

**Se procede a eliminar valores atipicos.**

In [ ]:
for c in df.columns:
    df[c] = df[c][np.abs(df[c]-df[c].mean())<=(2*df[c].std())]

df = df.dropna()
df.info()

In [ ]:
grafico_bloxplot(df['Volume_(BTC)'])
grafico_bloxplot(df['Volume_(Currency)'])
grafico_bloxplot(df['Weighted_Price'])

## <center> Normalización
**A continuación se procedera a escalar los valores entre 0 y 1**

In [ ]:
df_s = MinMaxScaler(feature_range=(0, 1))
df_s = df_s.fit_transform(df)

## <center> Separacion de data
**A continuación se procedera a separar la data, para entrenamiento, test y validacion. Posteriormente se probaran los distintos algoritmos de Machine Learning**

In [ ]:
df1 = pd.DataFrame(df_s,columns=['Volume_(BTC)', 'Volume_(Currency)', 'Weighted_Price'])
df2=df1

In [ ]:
df1=df1.sample(frac=1)
X=df1.drop(['Weighted_Price'],axis=1)
Y=df1['Weighted_Price']

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.80,shuffle=False)
X_train,X_val,Y_train,Y_val=train_test_split(X_train,Y_train,train_size=0.90,shuffle=False)
print('La data original contiene:', X.shape[0],'filas y',X.shape[1]+1,'columnas')
print('La data para entrenamiento contiene:', X_train.shape[0],'filas y',X_train.shape[1],'columnas')
print('La data de validación contiene:', X_val.shape[0],'filas y',X_val.shape[1],'columnas')
print('La data de test contiene:', X_test.shape[0],'filas y',X_test.shape[1],'columnas')

## <center> Desarrollo de Neural Network

In [ ]:
model = Sequential()

model.add(Dense(300, input_dim=X_train.shape[1], activation='relu')) 
model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(30, input_dim=X_train.shape[1], activation='relu')) 
model.add(Dense(3, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(1)) 

model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=100, batch_size=100, validation_data=(X_val, Y_val), verbose=False)

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
X2=df2.drop(['Weighted_Price'],axis=1)
Y2=df2['Weighted_Price']
X_train2,X_test2,Y_train2,Y_test2=train_test_split(X2,Y2,train_size=0.80,shuffle=False)
X_train2,X_val2,Y_train2,Y_val2=train_test_split(X_train2,Y_train2,train_size=0.90,shuffle=False)
pred = model.predict(X_test2)
plt.figure(figsize=(20, 10))
plt.plot(Y_test2.values, marker='.',label='Test')
plt.plot(pred, marker='.',label='Pred')
plt.legend()
plt.show()

In [ ]:
error = sqrt(mean_squared_error(Y_test2,pred))
print('RMSE =',error)
from sklearn.metrics import r2_score
print('R2 score =',r2_score(Y_test2, pred))

## <center> Desarrollo de Random Forest
**Se buscan los mejores parametros de "n_estimators" y "max_depth" mediante GridSearch, posteriormente se puede observar la precision del algoritmo.**

In [ ]:
param={'n_estimators': range(100,1000,20),'max_depth': range(2,10)}

forest = RandomForestRegressor()
forest = GridSearchCV(forest,param_grid=param,scoring='r2',cv=5,return_train_score=True)
forest.fit(X_train, Y_train)

randomforest_score = forest.score(X_val, Y_val)

print('Forest Score: ', randomforest_score)

**Se realiza Cross Validation en el algoritmo de Random Forest.**

In [ ]:
results= cross_validate(forest, X,Y,return_train_score=True,cv=5)
results

In [ ]:
pred = forest.predict(X_test2)
plt.figure(figsize=(20, 10))
plt.plot(Y_test2.values, marker='.')
plt.plot(pred, marker='.' )
plt.show()

## <center> Desarrollo de KNN
**Se buscan los mejores parametros de "n_neighbors" mediante GridSearch, posteriormente se pueden observar las metricas de desempeño del algoritmo.**

In [ ]:
model_KNN = neighbors.KNeighborsRegressor()
model_KNN=GridSearchCV(model_KNN,param_grid={'n_neighbors': list(range(2,20))})
model_KNN.fit(X_train, Y_train)

pred=model_KNN.predict(X_test)

error = sqrt(mean_squared_error(Y_test,pred))
print('RMSE value' , error)

coeff = np.corrcoef(Y_test,pred)
print('coeff value', coeff)

**Se realiza Cross Validation en el algoritmo KNN.**

In [ ]:
results= cross_validate(model_KNN, X,Y,return_train_score=True,cv=5)
results

In [ ]:
pred = model_KNN.predict(X_test2)
plt.figure(figsize=(20, 10))
plt.plot(Y_test2.values, marker='.')
plt.plot(pred, marker='.' )
plt.show()

## <center> Conclusion
**A traves de los diferentes algoritmos de regresión y considerando un dataset no sucesivo para entrenar tales algoritmos los resultados de la red neuronal y knn fueron sobresalientes. Lo cual hace concluir que se logra una buena estimación del valor del bitcoin a traves de 2 variables, en este caso Volume(BTC) y Volume(Currency)**
    
**Debido a que Cross-validation arrojo una muy buena puntuación y similar para los datos de entrenamiento y test no se puede considerar un overfitting en el algortimo. Para el caso de la NN las metricas R2 y RMSE son bastantes buenas las cuales indican que el modelo tiene un alto grado de precision y un buen ajuste con respecto a los datos**

## Extra

En la presente sección se realizara la predicción del precio del bitcoin solo con valores pasados de volumen y precio. Lo que podria ser de vital importancia al momento de querer estimar si es rentable una inversión en este tipo de criptomoneda

In [ ]:
def df_pasado(data, pasado, presente, dropnan=True):
    variables = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    columnas, nombre = list(), list()
    for i in range(pasado, 0, -1):
        columnas.append(df.shift(i))
        nombre += [('Variable%d(t-%d)' % (j+1, i)) for j in range(variables)]
    for i in range(0, presente):
        columnas.append(df.shift(-i))
        if i == 0:
            nombre += [('Variable%d(t)' % (j+1)) for j in range(variables)]
        else:
            nombre += [('Variable%d(t+%d)' % (j+1, i)) for j in range(variables)]
    matriz = concat(columnas, axis=1)
    matriz.columns = nombre
    if dropnan:
        matriz.dropna(inplace=True)
    return matriz

In [ ]:
df3 = df_pasado(df2, 2, 1)

In [ ]:
df3=df3.drop(['Variable1(t)', 'Variable2(t)'],axis=1)

In [ ]:
df3.columns=['Volume_(BTC)(t-2)', 'Volume_(Currency)(t-2)', 'Weighted_Price(t-2)', 'Volume_(BTC)(t-1)', 'Volume_(Currency)(t-1)',
       'Weighted_Price(t-1)', 'Weighted_Price(t)']

In [ ]:
df4=df3.sample(frac=1)
X=df4.drop(['Weighted_Price(t)'],axis=1)
Y=df4['Weighted_Price(t)']

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.80,shuffle=False)
X_train,X_val,Y_train,Y_val=train_test_split(X_train,Y_train,train_size=0.90,shuffle=False)
print('La data original contiene:', X.shape[0],'filas y',X.shape[1]+1,'columnas')
print('La data para entrenamiento contiene:', X_train.shape[0],'filas y',X_train.shape[1],'columnas')
print('La data de validación contiene:', X_val.shape[0],'filas y',X_val.shape[1],'columnas')
print('La data de test contiene:', X_test.shape[0],'filas y',X_test.shape[1],'columnas')

In [ ]:
model = Sequential()

model.add(Dense(300, input_dim=X_train.shape[1], activation='relu')) 
model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(30, input_dim=X_train.shape[1], activation='relu')) 
model.add(Dense(3, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(1)) 

model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=100, batch_size=100, validation_data=(X_val, Y_val), verbose=False)

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
X2=df3.drop(['Weighted_Price(t)'],axis=1)
Y2=df3['Weighted_Price(t)']
X_train2,X_test2,Y_train2,Y_test2=train_test_split(X2,Y2,train_size=0.80,shuffle=False)
X_train2,X_val2,Y_train2,Y_val2=train_test_split(X_train2,Y_train2,train_size=0.90,shuffle=False)
pred = model.predict(X_test2)
plt.figure(figsize=(20, 10))
plt.plot(Y_test2.values, marker='.',label='Test')
plt.plot(pred, marker='.',label='Pred')
plt.legend()
plt.show()

In [ ]:
error = sqrt(mean_squared_error(Y_test2,pred))
print('RMSE =',error)
from sklearn.metrics import r2_score
print('R2 score =',r2_score(Y_test2, pred))

# Conclusión 2

Debido a que el dataset es simple y las variables utilizadas afectan directamente el precio del bitcoin, los resultados de la predicción son buenos, y esto considerando solamente valores anteriores de las variables. Lo cual puede ser de gran aporte para decidir si invertir o no.